In [312]:
import pickle
import treeUtil
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from textblob import TextBlob
from textblob import Word
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from readability import Readability
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import string
import tensorflow as tf
import keras
from keras.models import load_model

In [313]:
[lib, con, neutral] = pickle.load(open('ibcData.pkl', 'rb'))

Extracting the leaf nodes

In [314]:
# leaf = lib[0].get_leaves()
# print(len(leaf))
# leaf[3].print_leaf()

# how to access sentence text

In [315]:
# print ('Liberal examples (out of ', len(lib), ' sentences): ')
# for tree in lib[0:5]:
#     print (tree.get_words())

In [316]:
# print ('\nConservative examples (out of ', len(con), ' sentences): ')
# for tree in con[0:5]:
#     print (tree.get_words())

In [317]:
# print ('\nNeutral examples (out of ', len(neutral), ' sentences): ')
# for tree in neutral[0:5]:
#     print (tree.get_words())

In [318]:
def listFromTree(tree):
    lst = []
    for ex_tree in tree:
        lst.append(ex_tree.get_words())
#        for node in ex_tree:

#         # remember, only certain nodes have labels (see paper for details)
#         if hasattr(node, 'label'):
#             if node.label == "Liberal":
#                 liberal.append(node.get_words())
#             elif node.label == "Neutral":
#                 neutral.append(node.get_words())
#             elif node.label == "Conservative":
#                 conserv.append(node.get_words())
#                 (node.label, ': ', node.get_words())
#     #             print (type(node.get_words()))
    return lst
        

In [455]:
def listsFromTrees(treelib, treeneu, treecon):
    lstLib = []
    lstNeu = []
    lstCon = []
    wordLen = 6
    for ex_tree in treelib:
        #lst.append(ex_tree.get_words())
        for node in ex_tree:
        # remember, only certain nodes have labels (see paper for details)
            if hasattr(node, 'label') and len(node.get_words().split()) > wordLen:
                if node.label == "Liberal":
                    lstLib.append(node.get_words())
                elif node.label == "Neutral":
                    lstNeu.append(node.get_words())
                elif node.label == "Conservative":
                    lstCon.append(node.get_words())

    for ex_tree in treeneu:
        #lst.append(ex_tree.get_words())
        for node in ex_tree:
        # remember, only certain nodes have labels (see paper for details)
            if hasattr(node, 'label') and len(node.get_words().split()) > wordLen:
                if node.label == "Liberal":
                    lstLib.append(node.get_words())
                elif node.label == "Neutral":
                    lstNeu.append(node.get_words())
                elif node.label == "Conservative":
                    lstCon.append(node.get_words())
                
    for ex_tree in treecon:
        #lst.append(ex_tree.get_words())
        for node in ex_tree:
        # remember, only certain nodes have labels (see paper for details)
            if hasattr(node, 'label') and len(node.get_words().split()) > wordLen:
                if node.label == "Liberal":
                    lstLib.append(node.get_words())
                elif node.label == "Neutral":
                    lstNeu.append(node.get_words())
                elif node.label == "Conservative":
                    lstCon.append(node.get_words())
    return lstLib, lstNeu, lstCon

In [512]:
# how to access phrase labels for a particular tree
#ex_tree = lib[1]

liberal = []
neut = []
conserv = []

#Only sentences
liberal = listFromTree(lib)
neut = listFromTree(neutral)
conserv = listFromTree(con)

#Sentences and phrases
#liberal, neut, conserv = listsFromTrees(lib, neutral, con)

print (len(liberal))
print (len(neut))
print (len(conserv))




2025
600
1701


In [513]:
def cleanList(dataList):
    newList = []
    #lowercase
    newList = [x.lower() for x in dataList]
    #removes punctuations
    newList = [s.translate(str.maketrans('', '', string.punctuation)) for s in  newList]
    #removes stop words
    stop = stopwords.words('english')
    for i in range(0, len(newList)):
        newList[i] = " ".join(x for x in newList[i].split() if x not in stop)
    return newList

In [514]:
liberal = cleanList(liberal)
neut = cleanList(neut)
conserv = cleanList(conserv)
print (len(liberal))
print (len(neut))
print (len(conserv))

2025
600
1701


In [515]:
l = pd.DataFrame({"liberal":liberal})
n = pd.DataFrame({"neutral":neut})
c = pd.DataFrame({"conservative":conserv})

In [516]:
data = pd.concat([l,n,c], axis=1)
data['liberal'] = data['liberal'].fillna('')
data['neutral'] = data['neutral'].fillna('')
data['conservative'] = data['conservative'].fillna('')

In [517]:
def topTen(dataFr, index):
    frequency = pd.Series(' '.join(dataFr[index]).split()).value_counts()[:10]
    frequency = list(frequency.index)
    return dataFr[index].apply(lambda x: " ".join(x for x in x.split() if x not in frequency))

In [518]:
def bottomTen(dataFr, index):
    frequency = pd.Series(' '.join(dataFr[index]).split()).value_counts()[-10:]
    frequency = list(frequency.index)
    return dataFr[index].apply(lambda x: " ".join(x for x in x.split() if x not in frequency))

In [519]:
#Removes top 10 words
data['neutral'] = topTen(data, "neutral")
data['liberal'] = topTen(data, "liberal")
data['conservative'] = topTen(data, "conservative")

In [520]:
#Removes top 10 rare words
data['neutral'] = bottomTen(data, "neutral")
data['liberal'] = bottomTen(data, "liberal")
data['conservative'] = bottomTen(data, "conservative")

In [521]:
def lemmat(dataFr, index):
    newData =dataFr
    for i in range(0, len(dataFr[index])):
        newData[index][i] = " ".join([Word(word).lemmatize() for word in dataFr[index][i].split()])
    return newData[index]

In [522]:
#Lemmatization
data['neutral'] = lemmat(data, "neutral")
data['liberal'] = lemmat(data, "liberal")
data['conservative'] = lemmat(data, "conservative")


In [523]:
#for col in data.columns:
all_text = " ".join(x for col in data.columns for x in data[col])
words = all_text.split()
len(words)

88949

In [524]:
from collections import Counter
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

polit_ints = []
for col in data.columns:
    for each in data[col]:
        if each != "":
            polit_ints.append([vocab_to_int[word] for word in each.split()])

In [525]:
print(len(polit_ints))

4326


In [526]:
x_train = []
y_train = []

# x_train = x_train + [x for x in data["liberal"]] 
# x_train = x_train + [x for x in data["neutral"][0:600]]
# x_train = x_train + [x for x in data["conservative"][0:1701]]

y_train = y_train + ([1] * len(liberal))
y_train = y_train + ([0] * len(neut))
y_train = y_train + ([-1] * len(conserv))



In [527]:
labels = np.array(y_train)

In [528]:
from collections import Counter
polit_lens = Counter([len(x) for x in polit_ints])
print("Zero-length sentences: {}".format(polit_lens[0]))
print("Maximum sentence length: {}".format(max(polit_lens)))

Zero-length sentences: 0
Maximum sentence length: 50


In [529]:
seq_len = 50
features = np.zeros((len(polit_ints), seq_len), dtype=int)
# print(features[:10,:100])
for i, row in enumerate(polit_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]
features[:10,:100]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0, 1389,  438,   20,  581,  377,  137,   54, 3292,   43,  539,
         110, 7108, 1039,  300, 1143,  196],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0, 3293,  802,  144, 5015,
        2802, 1288, 1040,  413, 2164, 5016,  841,  131, 1390,  100,  248,
         157, 1289, 3914, 1290, 3294,  561],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,  582, 5017,  329,  193,  104,  193,  145,
        3915,   20, 71

In [530]:
print(len(features))
print(len(labels))
print(type(features))
print(features[41])
print(len(features[41]))
print(polit_ints[41])
print(len(polit_ints[41]))
print()
for item in features: 
    if item[49] == 0:
        print( item)


4326
4326
<class 'numpy.ndarray'>
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0   17  616  940  849  491 1210  510  993  204  940 2822  123  438
    1 5059 5060   22 3921  193 3931 3932]
50
[17, 616, 940, 849, 491, 1210, 510, 993, 204, 940, 2822, 123, 438, 1, 5059, 5060, 22, 3921, 193, 3931, 3932]
21



In [531]:

features

array([[    0,     0,     0, ...,   300,  1143,   196],
       [    0,     0,     0, ...,  1290,  3294,   561],
       [    0,     0,     0, ...,   583,    73,    80],
       ...,
       [    0,     0,     0, ..., 14008,  2603,  2430],
       [    0,     0,     0, ...,   188,   259,  1883],
       [    0,     0,     0, ...,  2818,  7058,  1550]])

In [532]:
split_frac = 0.8


len_lib = len(liberal)
len_neu = len(neut)
len_con =  len(conserv)


split_index_lib = int(split_frac * len(liberal))
split_index_neu = int(len_lib + int(split_frac * len(neut)))
split_index_con =int(len_lib + len_neu + int(split_frac * len(conserv)))



split_frac = 0.5

size_lib = len_lib-split_index_lib
size_neu = len_lib+len_neu-split_index_neu
size_con = len(features)-split_index_con


split_index_lib = int(split_frac * size_lib)
split_index_neu = int(size_lib + int(split_frac * size_neu))
split_index_con = int(size_lib + size_neu + int(split_frac * size_con))

print(size_lib)
print(size_neu)
print(size_con)
print(len(features))
print()



print(split_index_lib)
print(split_index_neu)
print(split_index_con)

405
120
341
4326

202
465
695


In [533]:
len(conserv)

1701

In [534]:
split_frac = 0.8


len_lib = len(liberal)
len_neu = len(neut)
len_con =  len(conserv)


split_index_lib = int(split_frac * len(liberal))
split_index_neu = int(len_lib + int(split_frac * len(neut)))
split_index_con =int(len_lib + len_neu + int(split_frac * len(conserv)))


# 2025
# 600
# 1701

# 7809
# 8843
# 5969


train_x = np.array(features[:split_index_lib].tolist() + features[len_lib:split_index_neu].tolist() + features[len_lib+len_neu:split_index_con].tolist())
var_x = np.array(features[split_index_lib:len_lib].tolist() + features[split_index_neu:len_lib+len_neu].tolist() +   features[split_index_con:].tolist()) 
train_y = np.array(labels[:split_index_lib].tolist() + labels[len_lib:split_index_neu].tolist() + labels[len_lib+len_neu:split_index_con].tolist())
var_y = np.array(labels[split_index_lib:len_lib].tolist() + labels[split_index_neu:len_lib+len_neu].tolist() +   labels[split_index_con:].tolist()) 


split_frac = 0.5

size_lib = len_lib-split_index_lib
size_neu = len_lib+len_neu-split_index_neu
size_con = len(features)-split_index_con


split_index_lib = int(split_frac * size_lib)
split_index_neu = int(size_lib + int(split_frac * size_neu))
split_index_con = int(size_lib + size_neu + int(split_frac * size_con))


val_x, test_x = np.array(var_x[:split_index_lib+1].tolist() + var_x[size_lib:split_index_neu].tolist() + var_x[size_lib+size_neu:split_index_con].tolist()), np.array(var_x[split_index_lib+1:size_lib].tolist() + var_x[split_index_neu:size_lib+size_neu].tolist() + var_x[split_index_con:].tolist()) 
val_y, test_y = np.array(var_y[:split_index_lib+1].tolist() + var_y[size_lib:split_index_neu].tolist() + var_y[size_lib+size_neu:split_index_con].tolist()), np.array(var_y[split_index_lib+1:size_lib].tolist() + var_y[split_index_neu:size_lib+size_neu].tolist() + var_y[split_index_con:].tolist()) 

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))
print("label set: \t\t{}".format(train_y.shape), 
      "\nValidation label set: \t{}".format(val_y.shape),
      "\nTest label set: \t\t{}".format(test_y.shape))

			Feature Shapes:
Train set: 		(3460, 50) 
Validation set: 	(433, 50) 
Test set: 		(433, 50)
label set: 		(3460,) 
Validation label set: 	(433,) 
Test label set: 		(433,)


In [535]:
len(features)

4326

In [536]:
lstm_size = 256
lstm_layers = 2
batch_size = 1000
learning_rate = 0.01

In [537]:
n_words

14013

In [538]:
n_words = len(vocab_to_int) + 1 # Add 1 for 0 added to vocab
# Create the graph object
# tf.reset_default_graph()
# with tf.name_scope('inputs'):
#     inputs_ = tf.placeholder(tf.int32, [None, None], name="inputs")
#     labels_ = tf.placeholder(tf.int32, [None, None], name="labels")
#     keep_prob = tf.placeholder(tf.float32, name="keep_prob")

In [505]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.backend import clear_session

clear_session()
embedding_size=32
model=Sequential()
model.add(Embedding(n_words, embedding_size, input_length=50))
model.add(LSTM(100))
model.add(Dense(1, activation='tanh'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 32)            448416    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 501,717
Trainable params: 501,717
Non-trainable params: 0
_________________________________________________________________
None


In [506]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [507]:
model.fit(train_x, train_y, validation_data=(val_x, val_y), batch_size=32, epochs=50)

Train on 13563 samples, validate on 1697 samples
Epoch 1/50
13563/13563 [==============================] - 27s 2ms/step - loss: 1.3896 - acc: 0.3056 - val_loss: 1.6337 - val_acc: 0.3194
Epoch 2/50
13563/13563 [==============================] - 28s 2ms/step - loss: 1.6210 - acc: 0.3197 - val_loss: 1.6337 - val_acc: 0.3194
Epoch 3/50
13563/13563 [==============================] - 26s 2ms/step - loss: 1.6210 - acc: 0.3197 - val_loss: 1.6337 - val_acc: 0.3194
Epoch 4/50
13563/13563 [==============================] - 32s 2ms/step - loss: 1.6210 - acc: 0.3197 - val_loss: 1.6337 - val_acc: 0.3194
Epoch 5/50
13563/13563 [==============================] - 25s 2ms/step - loss: 1.6210 - acc: 0.3197 - val_loss: 1.6337 - val_acc: 0.3194
Epoch 6/50
13563/13563 [==============================] - 26s 2ms/step - loss: 1.6210 - acc: 0.3197 - val_loss: 1.6337 - val_acc: 0.3194
Epoch 7/50
13563/13563 [==============================] - 26s 2ms/step - loss: 1.6210 - acc: 0.3197 - val_loss: 1.6337 - val_acc:

In [508]:
scores = model.evaluate(test_x, test_y, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.31957547169811323


In [509]:
type(model)

keras.engine.sequential.Sequential

In [510]:

model.save('models/modelwPhr_tan_b32_e50.h5')


In [542]:
del model

# Best Model So Far

In [543]:
model = load_model('models/model_tan_b64_e100.h5')

In [544]:
scores = model.evaluate(test_x, test_y, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.45958429643793963
